# List of Exposures with Hologram


- work with Weakly_2022_34
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_34/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/spectractor/g90263d33d7/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_lsst/g61e05cf539+a8371b4bcb/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/gf10b05e212+a1470f6de0/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/shapelet/gd877ba84e5+2f58e93c51/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_modelfit/gf819d2c871+5816c8c9ab/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_extensions_shapeHSM/g6072bfb783+0a325c68f9/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet_extensions/g9d18589735+cc492336a9/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/proxmin/g33

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_34/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_34
   g0b29ad24fb+897c4bc163 	current w_2022_34 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [9]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022062800004,AT_O_20220628_000004,20220628,4,bias,Park position,SDSSr~empty,NaN,0.0,NaN,NaN,NaN
1,2022070600001,AT_O_20220706_000001,20220706,1,test,UNKNOWN,unknown~unknown,NaN,5.0,NaN,NaN,NaN
2,2022070700001,AT_O_20220707_000001,20220707,1,bias,UNKNOWN,empty~ronchi170lpmm,NaN,0.0,NaN,NaN,NaN
3,2022071100001,AT_O_20220711_000001,20220711,1,bias,UNKNOWN,FELH0600~holo4_003,NaN,0.0,NaN,NaN,NaN
4,2022071200001,AT_O_20220712_000001,20220712,1,bias,UNKNOWN,SDSSg~holo4_003,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44232,2022060900935,AT_O_20220609_000935,20220609,935,cwfs,HD 221054,SDSSg~empty,21.707667,30.0,352.285362,-51.019608,269.453648
44233,2022060900936,AT_O_20220609_000936,20220609,936,cwfs,HD 221054,SDSSg~empty,21.670582,30.0,352.28535,-51.019575,269.453634
44234,2022060900937,AT_O_20220609_000937,20220609,937,cwfs,HD 221054,SDSSg~empty,21.594164,30.0,352.285373,-51.019592,269.453635
44235,2022060900938,AT_O_20220609_000938,20220609,938,cwfs,HD 221054,SDSSg~empty,21.559853,30.0,352.285369,-51.019605,269.453759


## Selection of science

In [12]:
df_science = df_exposure[df_exposure.type == 'science']

In [13]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_12210/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [14]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
2065,2022063000741,AT_O_20220630_000741,20220630,741,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.216657,30.0,309.341775,-5.594321,50.470185
2066,2022063000742,AT_O_20220630_000742,20220630,742,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.311557,30.0,309.341675,-5.594353,50.472142
2067,2022063000743,AT_O_20220630_000743,20220630,743,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.36748,30.0,309.436196,-5.594359,50.230394
2068,2022063000744,AT_O_20220630_000744,20220630,744,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.459853,30.0,309.436102,-5.594346,50.231639
2069,2022063000745,AT_O_20220630_000745,20220630,745,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.571891,30.0,328.698147,17.412737,83.126668
2070,2022063000746,AT_O_20220630_000746,20220630,746,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.597445,30.0,328.698026,17.412692,83.124157
2071,2022063000747,AT_O_20220630_000747,20220630,747,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.759167,30.0,328.50127,17.506591,82.501275
2072,2022063000748,AT_O_20220630_000748,20220630,748,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.786121,30.0,328.501103,17.506714,82.502325
2073,2022063000749,AT_O_20220630_000749,20220630,749,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.801233,30.0,328.599795,17.506644,82.299976
2074,2022063000750,AT_O_20220630_000750,20220630,750,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.82869,30.0,328.599615,17.506728,82.298997


In [15]:
df_science.day_obs.unique()

array([20200127, 20200128, 20200129, 20200212, 20200217, 20200218,
       20200219, 20200220, 20200221, 20200312, 20200313, 20200314,
       20200315, 20200316, 20210119, 20210120, 20210121, 20210126,
       20210209, 20210216, 20210217, 20210218, 20210309, 20210310,
       20210311, 20210323, 20210524, 20210525, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210727, 20210804,
       20210817, 20210907, 20210908, 20210909, 20211005, 20211006,
       20211102, 20211103, 20211104, 20211130, 20220208, 20220209,
       20220215, 20220216, 20220217, 20220308, 20220311, 20220316,
       20220317, 20220405, 20220406, 20220407, 20220503, 20220504,
       20220505, 20220524, 20220525, 20220526, 20220607, 20220608,
       20220609, 20220628, 20220629, 20220630])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [16]:
date_sel = 20210909

In [17]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [18]:
len(df_science_selected)

614

# List of filters

In [19]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['FELH0600~empty', 'empty~ronchi170lpmm', 'empty~holo4_003',
       'BG40~empty', 'FELH0600~ronchi170lpmm', 'BG40~ronchi170lpmm',
       'quadnotch1~ronchi170lpmm', 'BG40~holo4_003', 'FELH0600~holo4_003',
       'quadnotch1~holo4_003', 'BG40~ronchi90lpmm', 'empty~ronchi90lpmm',
       'quadnotch1~ronchi90lpmm', 'FELH0600~ronchi90lpmm'], dtype=object)

In [20]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003',
       'quadnotch1~holo4_003'], dtype='<U20')

## Selection of filter

- But now we have a loop on filters

In [21]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20210909_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20210909_filt_BG40-holo4_003.list
FELH0600~holo4_003
visitdispersers_20210909_filt_FELH0600-holo4_003.list
quadnotch1~holo4_003
visitdispersers_20210909_filt_quadnotch1-holo4_003.list


/tmp/ipykernel_12210/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_12210/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_12210/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [24]:
len(all_def_science_selected_final)

4

In [25]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
21409,2021090900165,AT_O_20210909_000165,20210909,165,science,PNG321.0+03.9,empty~holo4_003,43.622901,10.0,224.94839,-54.259753,211.751528
21410,2021090900166,AT_O_20210909_000166,20210909,166,science,PNG321.0+03.9,empty~holo4_003,43.677652,10.0,224.948414,-54.25972,211.896527
21411,2021090900167,AT_O_20210909_000167,20210909,167,science,PNG321.0+03.9,empty~holo4_003,43.732445,10.0,224.948405,-54.259693,212.041472
21412,2021090900168,AT_O_20210909_000168,20210909,168,science,PNG321.0+03.9,empty~holo4_003,43.787273,30.0,224.948365,-54.259662,212.186374
21413,2021090900169,AT_O_20210909_000169,20210909,169,science,PNG321.0+03.9,empty~holo4_003,43.88974,30.0,224.948427,-54.259583,212.456842
21414,2021090900170,AT_O_20210909_000170,20210909,170,science,PNG321.0+03.9,empty~holo4_003,43.992326,30.0,224.9484,-54.259526,212.727095
21415,2021090900171,AT_O_20210909_000171,20210909,171,science,PNG321.0+03.9,empty~holo4_003,44.094978,60.0,224.948376,-54.259488,212.997003
21417,2021090900175,AT_O_20210909_000175,20210909,175,science,PNG321.0+03.9,empty~holo4_003,45.359845,60.0,224.948243,-54.258801,216.285394
21418,2021090900176,AT_O_20210909_000176,20210909,176,science,PNG321.0+03.9,empty~holo4_003,45.535352,60.0,224.94819,-54.258719,216.736441
21419,2021090900177,AT_O_20210909_000177,20210909,177,science,PNG321.0+03.9,empty~holo4_003,45.713387,60.0,224.948154,-54.258636,217.192765


In [26]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
21594,2021090900372,AT_O_20210909_000372,20210909,372,science,HD 185975,BG40~holo4_003,57.162034,20.0,306.29005,-87.49945,164.381559
21602,2021090900388,AT_O_20210909_000388,20210909,388,science,HD 200654,BG40~holo4_003,20.87836,20.0,316.629319,-49.984796,135.076607
21610,2021090900401,AT_O_20210909_000401,20210909,401,science,HD 009051,BG40~holo4_003,47.404968,20.0,22.236013,-24.349415,27.734594
21617,2021090900443,AT_O_20210909_000443,20210909,443,science,HD 185975,BG40~holo4_003,57.468491,20.0,306.179523,-87.484545,182.620991
21623,2021090900453,AT_O_20210909_000453,20210909,453,science,HD 200654,BG40~holo4_003,27.413591,20.0,316.615043,-49.971815,171.673191
21629,2021090900462,AT_O_20210909_000462,20210909,462,science,HD 009051,BG40~holo4_003,33.692252,20.0,22.223107,-24.344564,15.036545
21635,2021090900479,AT_O_20210909_000479,20210909,479,science,HD 185975,BG40~holo4_003,57.722448,20.0,306.162041,-87.477573,192.475522
21641,2021090900489,AT_O_20210909_000489,20210909,489,science,HD 200654,BG40~holo4_003,32.792946,20.0,316.608612,-49.965273,189.66424
21647,2021090900498,AT_O_20210909_000498,20210909,498,science,HD 009051,BG40~holo4_003,25.300478,20.0,22.215711,-24.34165,5.513844
21653,2021090900508,AT_O_20210909_000508,20210909,508,science,HD 185975,BG40~holo4_003,57.941197,20.0,306.146966,-87.47261,199.660924


In [27]:
all_def_science_selected_final[2]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
22205,2021090900373,AT_O_20210909_000373,20210909,373,science,HD 185975,FELH0600~holo4_003,57.163764,20.0,306.287227,-87.499447,164.5363
21917,2021090900389,AT_O_20210909_000389,20210909,389,science,HD 200654,FELH0600~holo4_003,20.917052,20.0,316.628785,-49.984944,135.451972
22190,2021090900402,AT_O_20210909_000402,20210909,402,science,HD 009051,FELH0600~holo4_003,47.279455,20.0,22.23593,-24.349327,27.628017
2174,2021090900444,AT_O_20210909_000444,20210909,444,science,HD 185975,FELH0600~holo4_003,57.472117,20.0,306.176793,-87.484523,182.776982
21981,2021090900454,AT_O_20210909_000454,20210909,454,science,HD 200654,FELH0600~holo4_003,27.487886,20.0,316.614673,-49.971785,171.959119
22525,2021090900463,AT_O_20210909_000463,20210909,463,science,HD 009051,FELH0600~holo4_003,33.565921,20.0,22.222977,-24.344517,14.906628
2159,2021090900464,AT_O_20210909_000464,20210909,464,science,HD 009051,FELH0600~holo4_003,32.595635,2.0,22.194213,-24.341136,13.839019
22663,2021090900480,AT_O_20210909_000480,20210909,480,science,HD 185975,FELH0600~holo4_003,57.727006,20.0,306.158981,-87.477569,192.632312
22378,2021090900490,AT_O_20210909_000490,20210909,490,science,HD 200654,FELH0600~holo4_003,32.879608,20.0,316.608328,-49.965223,189.920226
21921,2021090900499,AT_O_20210909_000499,20210909,499,science,HD 009051,FELH0600~holo4_003,25.175119,20.0,22.215671,-24.341636,5.35353


In [28]:
all_def_science_selected_final[3]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
21595,2021090900374,AT_O_20210909_000374,20210909,374,science,HD 185975,quadnotch1~holo4_003,57.165569,10.0,306.284026,-87.499455,164.695799
21603,2021090900390,AT_O_20210909_000390,20210909,390,science,HD 200654,quadnotch1~holo4_003,20.957401,10.0,316.628317,-49.985146,135.83727
21611,2021090900403,AT_O_20210909_000403,20210909,403,science,HD 009051,quadnotch1~holo4_003,47.150364,10.0,22.235886,-24.349281,27.518281
21675,2021090900563,AT_O_20210909_000563,20210909,563,science,HD 2811,quadnotch1~holo4_003,13.416537,15.0,7.807733,-43.628944,115.267271
21676,2021090900564,AT_O_20210909_000564,20210909,564,science,HD 2811,quadnotch1~holo4_003,13.434197,15.0,7.807523,-43.629182,115.7112
21816,2021090900757,AT_O_20210909_000757,20210909,757,science,HD 009051,quadnotch1~holo4_003,22.530971,10.0,22.176368,-24.32567,223.482223
21824,2021090900770,AT_O_20210909_000770,20210909,770,science,HD 031128,quadnotch1~holo4_003,20.099624,10.0,73.066602,-27.065631,5.608253
21832,2021090900786,AT_O_20210909_000786,20210909,786,science,HD 185975,quadnotch1~holo4_003,60.425049,10.0,306.570604,-87.429368,260.582009


In [29]:
all_def_science_selected_final[4]

IndexError: list index out of range